# Flight Dashboard

## Import Libraries

In [1]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [4]:


# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

# Create a dash application
app = dash.Dash(__name__)

# Build dash app layout
app.layout = html.Div(children=[ html.H1('Flight Detail Statistics Dashboard',style={'textAlign': 'right', 'color': '#503D36', 'font-size': 30}),
                                html.Div(["Input Year: ", dcc.Input(id='input-year',value='2010',type='number',style={'height':'35px', 'font_size':30})],
                                style={'font-size': 30}),
                                html.Br(),
                                html.Br(), 
                                html.Div([
                                        html.Div(dcc.Graph(id='carrier-plot')),
                                        html.Div(dcc.Graph(id='weather-plot'))
                                ], style={'display': 'flex'}),
    
                                html.Div([
                                        html.Div(dcc.Graph(id='nas-plot')),
                                        html.Div(dcc.Graph(id='security-plot'))
                                ], style={'display': 'flex'}),
                                
                                html.Div(dcc.Graph(id='late-plot'), style={'width':'100%', 'display': 'inline-block'})
                                ])

def compute_info(airline_data,entered_year):
    # Select the data from airline_data the csv imported
    df = airline_data[airline_data['Year']==int(entered_year)]

    # Compute delay averages for each graph
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_nas = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_security = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()

    return avg_car, avg_weather, avg_nas,avg_security,avg_late

@app.callback([
                Output(component_id='carrier-plot', component_property='figure'),
                Output(component_id='weather-plot', component_property='figure'),
                Output(component_id='nas-plot', component_property='figure'),
                Output(component_id='security-plot', component_property='figure'),
                Output(component_id='late-plot', component_property='figure')
                ],
                Input(component_id='input-year', component_property='value')
)

def get_graph(entered_year):
    # Compute required information for creating graph from the data
    avg_car, avg_weather,avg_nas,avg_security,avg_late = compute_info(airline_data, entered_year)
    
    # Line plot for carrier delay
    carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrier delay time (minutes) by airline')
    # Line plot for weather delay
    weather_fig = px.line(avg_weather, x='Month',y='WeatherDelay', color='Reporting_Airline', title='Average Weather Delay time (minutes) by airline')
    # Line plot for nas
    nas_figure = px.line(avg_nas, x='Month', y='NASDelay', color='Reporting_Airline', title='Average National Air System Delay time (minutes) by airline')
    # Line plot for Security
    security_figure = px.line(avg_security, x='Month', y='SecurityDelay', color = 'Reporting_Airline', title='Average Security Delay (minutes) by airline')
    # Line plot for late aircraft
    late_figure = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average Late Aircraft Delay (minutes) by airline')

    return[carrier_fig, weather_fig, nas_figure, security_figure, late_figure]

#Run the app
if __name__=='__main__':
    app.run_server()